In [203]:
import pandas as pd
from pathlib import Path

csv_path = Path.home() / "Desktop" / "Kingston Uni" / "student_outcomes_allProviders.csv"
defs_path = Path.home() / "Desktop" / "Kingston Uni" / "B3_TEF_Indicator_Data_Definitions.xlsx"

csv_path.exists(), defs_path.exists()



(True, True)

In [204]:
sample = pd.read_csv(csv_path, nrows=5)
print("Columns:", len(sample.columns))
sample.columns.tolist()


Columns: 69


['UKPRN_of_registering_provider',
 'POPULATION_TYPE',
 'INDICATOR_NAME',
 'MODE',
 'LEVEL',
 'SPLIT_IND_TYPE',
 'SPLIT_IND',
 'YEAR',
 'SUPP_REASON',
 'SUPP_REASON_INT_STUDY',
 'SUPP_REASON_SIG_INT_STUDY',
 'SUPP_REASON_BK',
 'INDICATOR_VALUE',
 'DENOMINATOR',
 'RESPONSE_RATE',
 'GO_NEG_INT_STUDY_RATE',
 'GO_NEG_SIG_INT_STUDY_RATE',
 'NUMERIC_THRESHOLD',
 'BELOW_THRESHOLD',
 'ABOVE_THRESHOLD',
 'BENCHMARK_VALUE',
 'DIFFERENCE',
 'AVG_CONTR_BENCHMARK',
 'BELOW_BENCH',
 'ABOVE_BENCH',
 'INDICATOR_LOWERCI99',
 'INDICATOR_LOWERCI97',
 'INDICATOR_LOWERCI95',
 'INDICATOR_LOWERCI92',
 'INDICATOR_LOWERCI90',
 'INDICATOR_LOWERCI87',
 'INDICATOR_LOWERCI85',
 'INDICATOR_LOWERCI82',
 'INDICATOR_LOWERCI80',
 'INDICATOR_LOWERCI77',
 'INDICATOR_LOWERCI75',
 'INDICATOR_UPPERCI99',
 'INDICATOR_UPPERCI97',
 'INDICATOR_UPPERCI95',
 'INDICATOR_UPPERCI92',
 'INDICATOR_UPPERCI90',
 'INDICATOR_UPPERCI87',
 'INDICATOR_UPPERCI85',
 'INDICATOR_UPPERCI82',
 'INDICATOR_UPPERCI80',
 'INDICATOR_UPPERCI77',
 'INDICA

In [205]:
CORE_COLS = [
    # identity / filters
    "UKPRN_of_registering_provider",
    "POPULATION_TYPE",
    "INDICATOR_NAME",
    "MODE",
    "LEVEL",
    "SPLIT_IND_TYPE",
    "SPLIT_IND",
    "YEAR",

    # values we chart
    "INDICATOR_VALUE",
    "BENCHMARK_VALUE",
    "DIFFERENCE",
    "BELOW_BENCH",
    "ABOVE_BENCH",

    # useful context / QA
    "DENOMINATOR",
    "SUPP_REASON",
    "NUMERIC_THRESHOLD",
    "BELOW_THRESHOLD",
    "ABOVE_THRESHOLD",
]


In [206]:
missing = [c for c in CORE_COLS if c not in sample.columns]
missing


[]

In [207]:
KINGSTON_UKPRN = 10003678

chunks = []
for chunk in pd.read_csv(csv_path, usecols=CORE_COLS, chunksize=200_000, low_memory=False):
    # make sure UKPRN is numeric (sometimes reads as text)
    chunk["UKPRN_of_registering_provider"] = pd.to_numeric(
        chunk["UKPRN_of_registering_provider"], errors="coerce"
    )
    
    # keep only Kingston rows
    filtered = chunk[chunk["UKPRN_of_registering_provider"] == KINGSTON_UKPRN]
    
    if not filtered.empty:
        chunks.append(filtered)

kingston = pd.concat(chunks, ignore_index=True)
kingston.shape


(3117, 18)

In [208]:
kingston["UKPRN_of_registering_provider"].value_counts()


UKPRN_of_registering_provider
10003678    3117
Name: count, dtype: int64

In [209]:
kingston["SPLIT_IND_TYPE"].value_counts().head(15)


SPLIT_IND_TYPE
SubjectOfStudy                   906
Ethnicity                        410
Year                             351
AgeOnCommencement                279
Disability                       190
Sex                              190
DeprivationQuintile              182
Domicile                         152
ABCSQuintile                     122
PartnershipType                  119
Overall                          101
GOQuintile                        84
CourseType_OtherUndergraduate     22
CourseType_FoundationYear          9
Name: count, dtype: int64

In [210]:
NUM_COLS = ["INDICATOR_VALUE", "BENCHMARK_VALUE", "DIFFERENCE", "DENOMINATOR"]

for c in NUM_COLS:
    kingston[c + "_NUM"] = pd.to_numeric(kingston[c], errors="coerce")

kingston[[*NUM_COLS, *(c + "_NUM" for c in NUM_COLS)]].head(10)


,INDICATOR_VALUE,BENCHMARK_VALUE,DIFFERENCE,DENOMINATOR,INDICATOR_VALUE_NUM,BENCHMARK_VALUE_NUM,DIFFERENCE_NUM,DENOMINATOR_NUM
0,82.4,82.4,0.0,50,82.4,82.4,0.0,50.0
1,89.2,88.0,1.2,150,89.2,88.0,1.2,150.0
2,NaN,96.1,[DPH],40,NaN,96.1,NaN,40.0
3,89.8,87.3,2.5,90,89.8,87.3,2.5,90.0
4,83.3,82.5,0.8,70,83.3,82.5,0.8,70.0
5,92.7,93.6,-1.0,80,92.7,93.6,-1.0,80.0
6,81.3,83.0,-1.6,90,81.3,83.0,-1.6,90.0
7,82.4,84.7,-2.3,90,82.4,84.7,-2.3,90.0
8,92.7,90.0,2.7,150,92.7,90.0,2.7,150.0
9,NaN,[low],[low],[low],NaN,NaN,NaN,NaN


In [211]:
kingston["INDICATOR_VALUE_NUM"].notna().mean()


0.6567212062880975

In [212]:
# How many rows have a denominator and what's typical size?
kingston["DENOMINATOR_NUM"].describe()


count    2015.000000
mean      203.955335
std       206.546262
min        20.000000
25%        60.000000
50%       120.000000
75%       270.000000
max       970.000000
Name: DENOMINATOR_NUM, dtype: float64

In [213]:
subj = kingston[
    (kingston["INDICATOR_NAME"] == "Continuation") &
    (kingston["SPLIT_IND_TYPE"] == "SubjectOfStudy") &
    (kingston["POPULATION_TYPE"] == "Taught") &
    (kingston["LEVEL"] == "AllUndergraduates")
].copy()

subj["DENOMINATOR_NUM"].describe()


count      4.000000
mean     130.000000
std      106.144556
min       50.000000
25%       57.500000
50%       95.000000
75%      167.500000
max      280.000000
Name: DENOMINATOR_NUM, dtype: float64

In [214]:
tmp = kingston[
    (kingston["INDICATOR_NAME"] == "Continuation") &
    (kingston["SPLIT_IND_TYPE"] == "SubjectOfStudy")
].copy()

tmp.shape


(304, 22)

In [215]:
tmp["POPULATION_TYPE"].value_counts()


POPULATION_TYPE
TaughtOrRegistered_(TorR)    142
Taught                       138
Partnership                   24
Name: count, dtype: int64

In [216]:
tmp["LEVEL"].value_counts()


LEVEL
PostgraduateTaughtMasters                  87
FirstDegree                                79
PostgraduateResearch                       70
OtherPostgraduate                          24
OtherUndergraduate                         20
AllUndergraduates                          11
UndergraduateWithPostgraduateComponents     8
PGCE                                        5
Name: count, dtype: int64

In [217]:
tmp["MODE"].value_counts()


MODE
Full-time         193
Part-time         100
Apprenticeship     11
Name: count, dtype: int64

In [218]:
tmp["YEAR"].value_counts()


YEAR
2019-2022    204
2018-2021    100
Name: count, dtype: int64

In [219]:
tmp.groupby(["POPULATION_TYPE", "LEVEL", "MODE", "YEAR"]).size().sort_values(ascending=False).head(20)


POPULATION_TYPE            LEVEL                      MODE            YEAR     
TaughtOrRegistered_(TorR)  FirstDegree                Full-time       2019-2022    28
Taught                     FirstDegree                Full-time       2019-2022    27
TaughtOrRegistered_(TorR)  PostgraduateTaughtMasters  Full-time       2019-2022    22
Taught                     PostgraduateTaughtMasters  Full-time       2019-2022    22
TaughtOrRegistered_(TorR)  PostgraduateResearch       Full-time       2019-2022    22
Taught                     PostgraduateResearch       Full-time       2019-2022    22
                           PostgraduateTaughtMasters  Part-time       2018-2021    21
TaughtOrRegistered_(TorR)  PostgraduateTaughtMasters  Part-time       2018-2021    21
Taught                     PostgraduateResearch       Part-time       2018-2021    13
TaughtOrRegistered_(TorR)  PostgraduateResearch       Part-time       2018-2021    13
Partnership                OtherUndergraduate         Full-t

In [220]:
kingston[kingston["INDICATOR_NAME"]=="Continuation"]["LEVEL"].value_counts()


LEVEL
FirstDegree                                207
PostgraduateTaughtMasters                  190
PostgraduateResearch                       158
OtherUndergraduate                         121
OtherPostgraduate                          112
PGCE                                        98
UndergraduateWithPostgraduateComponents     84
AllUndergraduates                           76
Name: count, dtype: int64

In [221]:
ug_all = kingston[
    (kingston["INDICATOR_NAME"] == "Continuation") &
    (kingston["LEVEL"] == "AllUndergraduates")
].copy()

ug_all.shape, ug_all["SPLIT_IND_TYPE"].value_counts().head(20)


((76, 22),
 SPLIT_IND_TYPE
 Ethnicity                        11
 SubjectOfStudy                   11
 Year                              9
 ABCSQuintile                      8
 AgeOnCommencement                 8
 Sex                               6
 DeprivationQuintile               5
 Disability                        5
 Domicile                          5
 Overall                           3
 PartnershipType                   3
 CourseType_OtherUndergraduate     2
 Name: count, dtype: int64)

In [222]:
allug = kingston[
    (kingston["INDICATOR_NAME"] == "Continuation") &
    (kingston["LEVEL"] == "AllUndergraduates")
].copy()

# keep only rows where the main value is published (suppressed rows become NaN in *_NUM)
allug_pub = allug[allug["INDICATOR_VALUE_NUM"].notna()].copy()

allug.shape, allug_pub.shape, allug_pub["POPULATION_TYPE"].value_counts()


((76, 22),
 (52, 22),
 POPULATION_TYPE
 Taught                       26
 TaughtOrRegistered_(TorR)    26
 Name: count, dtype: int64)

In [223]:
allug_pub["FOCUS_FLAG"] = pd.cut(
    allug_pub["DIFFERENCE_NUM"],
    bins=[-1e9, -1, 0, 1e9],
    labels=["Focus", "Watch", "OK"]
).astype("object")


In [224]:
allug_pub.loc[allug_pub["DIFFERENCE_NUM"].isna(), "FOCUS_FLAG"] = "Suppressed/NA"


In [225]:
allug_pub["COHORT_FLAG"] = pd.cut(
    allug_pub["DENOMINATOR_NUM"],
    bins=[0, 49, 999999],
    labels=["Small cohort (<50)", "OK (>=50)"]
).astype("object")


In [226]:
allug_pub[["POPULATION_TYPE","SPLIT_IND_TYPE","SPLIT_IND","DIFFERENCE_NUM","DENOMINATOR_NUM","FOCUS_FLAG","COHORT_FLAG"]].head(12)


,POPULATION_TYPE,SPLIT_IND_TYPE,SPLIT_IND,DIFFERENCE_NUM,DENOMINATOR_NUM,FOCUS_FLAG,COHORT_FLAG
465,Taught,ABCSQuintile,ABCSQ1,2.3,200.0,OK,OK (>=50)
466,Taught,ABCSQuintile,ABCSQ2_3,0.6,250.0,OK,OK (>=50)
467,Taught,ABCSQuintile,ABCSQ4_5,-6.2,70.0,Focus,OK (>=50)
468,Taught,AgeOnCommencement,Age21_30,-0.3,130.0,Watch,OK (>=50)
469,Taught,AgeOnCommencement,Age31andOver,3.7,290.0,OK,OK (>=50)
470,Taught,AgeOnCommencement,AgeUnder21,-8.9,100.0,Focus,OK (>=50)
471,Taught,CourseType_OtherUndergraduate,OtherUndergraduateLevel5+,-0.4,270.0,Watch,OK (>=50)
472,Taught,DeprivationQuintile,IMDQ1_2,1.1,230.0,OK,OK (>=50)
473,Taught,DeprivationQuintile,IMDQ3_5,0.0,290.0,Watch,OK (>=50)
474,Taught,Disability,Disabled,-2.6,80.0,Focus,OK (>=50)


In [227]:
allug_taught = allug_pub[allug_pub["POPULATION_TYPE"] == "Taught"].copy()


In [228]:
priority_taught = allug_taught.sort_values("DIFFERENCE_NUM").copy()


In [229]:
priority_taught[["SPLIT_IND_TYPE","SPLIT_IND","DIFFERENCE_NUM","DENOMINATOR_NUM","FOCUS_FLAG","COHORT_FLAG"]].head(15)


,SPLIT_IND_TYPE,SPLIT_IND,DIFFERENCE_NUM,DENOMINATOR_NUM,FOCUS_FLAG,COHORT_FLAG
470,AgeOnCommencement,AgeUnder21,-8.9,100.0,Focus,OK (>=50)
467,ABCSQuintile,ABCSQ4_5,-6.2,70.0,Focus,OK (>=50)
474,Disability,Disabled,-2.6,80.0,Focus,OK (>=50)
489,SubjectOfStudy,CAH26-01,-2.3,60.0,Focus,OK (>=50)
480,Ethnicity,Mixed,-1.2,40.0,Focus,Small cohort (<50)
491,Year,Year1,-0.8,140.0,Watch,OK (>=50)
482,Ethnicity,White,-0.8,270.0,Watch,OK (>=50)
471,CourseType_OtherUndergraduate,OtherUndergraduateLevel5+,-0.4,270.0,Watch,OK (>=50)
468,AgeOnCommencement,Age21_30,-0.3,130.0,Watch,OK (>=50)
490,SubjectOfStudy,CAH02-04,-0.2,280.0,Watch,OK (>=50)


In [230]:

out_dir = Path.home() / "Desktop" / "Kingston Uni" / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)

out_file = out_dir / "kingston_continuation_allug_monitoring_TAUGHT.csv"
priority_taught.to_csv(out_file, index=False)

out_file



PosixPath('/Users/user/Desktop/Kingston Uni/outputs/kingston_continuation_allug_monitoring_TAUGHT.csv')

In [231]:
import pandas as pd

# 1) Load the definitions table (it contains label mappings, incl. CAH subject codes)
defs = pd.read_excel(defs_path, sheet_name="Field_values")

# 2) Standardise the messy column names
defs = defs.rename(columns={
    defs.columns[0]: "FIELD",
    defs.columns[1]: "VALUE",
    defs.columns[2]: "DESC"
})

defs = defs[["FIELD", "VALUE", "DESC"]].dropna(subset=["FIELD", "VALUE"])

# 3) Build mapping dictionaries
split_ind_map = (
    defs[defs["FIELD"] == "SPLIT_IND"]
    .dropna(subset=["DESC"])
    .set_index("VALUE")["DESC"]
    .to_dict()
)

split_type_map = (
    defs[defs["FIELD"] == "SPLIT_IND_TYPE"]
    .dropna(subset=["DESC"])
    .set_index("VALUE")["DESC"]
    .to_dict()
)

# 4) Add friendly label columns to your Tableau dataset
priority_taught["SPLIT_IND_LABEL"] = priority_taught["SPLIT_IND"].map(split_ind_map).fillna(priority_taught["SPLIT_IND"])
priority_taught["SPLIT_IND_TYPE_LABEL"] = priority_taught["SPLIT_IND_TYPE"].map(split_type_map).fillna(priority_taught["SPLIT_IND_TYPE"])

# Nice combined label for Tableau rows
priority_taught["CATEGORY_LABEL"] = priority_taught["SPLIT_IND_TYPE_LABEL"] + ": " + priority_taught["SPLIT_IND_LABEL"]

# 5) Overwrite the SAME file you already connected to in Tableau
priority_taught.to_csv(out_file, index=False)

out_file



PosixPath('/Users/user/Desktop/Kingston Uni/outputs/kingston_continuation_allug_monitoring_TAUGHT.csv')

In [263]:

# Quick sanity checks: do we have the fields, and are values matching?

print("Defs FIELD unique (sample):", defs["FIELD"].dropna().unique()[:10])

print("\nExample SPLIT_IND values in your data (sample):")
print(priority_taught["SPLIT_IND"].dropna().astype(str).unique()[:15])

print("\nHow many SPLIT_IND values got a label?")
mapped = priority_taught["SPLIT_IND"].map(split_ind_map)
print(mapped.notna().mean(), "proportion mapped")

print("\nTop 20 unmapped SPLIT_IND values:")
unmapped = priority_taught.loc[mapped.isna(), "SPLIT_IND"].astype(str).value_counts().head(20)
print(unmapped)

print("\nExample mapped rows (where mapping worked):")
tmp = priority_taught.assign(_mapped=mapped)
print(tmp.loc[tmp["_mapped"].notna(), ["SPLIT_IND", "_mapped"]].head(15))


Defs FIELD unique (sample): ['Field name' 'UKPRN' 'POPULATION_TYPE' 'INDICATOR_NAME' 'MODE' 'LEVEL'
 'HIGH_BENCHMARK' 'SPLIT_IND_TYPE' 'SPLIT_IND' 'YEAR']

Example SPLIT_IND values in your data (sample):
['AgeUnder21' 'ABCSQ4_5' 'Disabled' 'CAH26-01' 'Mixed' 'Year1' 'White'
 'OtherUndergraduateLevel5+' 'Age21_30' 'CAH02-04' 'Year2' 'CAH10-01'
 'IMDQ3_5' 'Female' 'UK']

How many SPLIT_IND values got a label?
0.9615384615384616 proportion mapped

Top 20 unmapped SPLIT_IND values:
SPLIT_IND
Overall indicator    1
Name: count, dtype: int64

Example mapped rows (where mapping worked):
                     SPLIT_IND  \
470                 AgeUnder21   
467                   ABCSQ4_5   
474                   Disabled   
489                   CAH26-01   
480                      Mixed   
491                      Year1   
482                      White   
471  OtherUndergraduateLevel5+   
468                   Age21_30   
490                   CAH02-04   
492                      Year2   
486  